In [ ]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame, Panel

%pylab inline


In [ ]:
import os
main_folder = os.path.expanduser('~/Downloads/step_log')

In [ ]:
import re
regex_all_log = re.compile('\S*\.log{1}\.\d*')
regex_soll = re.compile('\S*control_output\.log{1}\.\d*')
regex_ist = re.compile('\S*task_vel\.log{1}\.\d*')

#l = ['this', 'is', 'just', 'a', 'test']
#matches = [string for string in l if re.match(regex, string)]

list_soll = []
list_ist = []

for root, dirs, files in os.walk(main_folder):
    for name in files:
        # Rename all *.log to *.log.0 (Not sure if this is a good idea after all)
        if name.endswith(".log"):
            print "RENAMING: {0} to {0}.0".format(name)
            full_path = os.path.join(root, name)
            os.rename(full_path, full_path+".0")
            name = name+".0"
        # Prepare list of all log files with path
        if re.match(regex_soll, name):
            list_soll.append(os.path.join(root, name))
        elif re.match(regex_ist, name):
            list_ist.append(os.path.join(root, name))
                        
print "Found {} IST log files".format(len(list_ist))
print "Found {} SOLL log files".format(len(list_soll))

#print list_soll

In [ ]:
def sortLogsToDfList(list_logs):
    regex = re.compile('(\S*\.(\d*))')
    matches = [re.findall(regex, string)[0] for string in list_logs]
    num = [int(tup_string[1]) for tup_string in matches]
    
    list_df = [pd.DataFrame()]*(max(num)+1)
    
    for match in matches:
        try:
            list_df[int(match[1])].append(
                pd.read_csv(match[0])
            )
        except Exception as e:
            print e.str()
    return list_df

In [ ]:
#list_df_soll = sortLogsToDfList(list_soll)
list_df_soll

In [ ]:
df_soll = pd.read_csv('~/Downloads/step_log/2018-01-08/2018-01-08_15-53-28_control_output.log.1')
df_soll.rename(columns=lambda x: x.strip(), inplace=True)

df_soll = df_soll.set_index('time')

In [ ]:
df_ist = pd.read_csv('~/Downloads/step_log/2018-01-08/2018-01-08_15-53-28_task_vel.log.1')
df_ist.rename(columns=lambda x: x.strip(), inplace=True)

df_ist = df_ist.set_index('time')

In [ ]:
df_step = pd.concat([df_soll['values[0]'], df_ist['values[2]']], axis=1).fillna(method='pad')
df_step.index = pd.to_datetime(df_step.index)

#pd.rolling_mean(df_step, 10, center=True).plot()
#df_step.rolling(window=10, center=True).mean().plot()

df_step.plot()